# Preprocesmaiento de datos INEGI

Limpieza, normalizacion, interpoalcion y extrapolacion de datos de la INEGI 


# Import

In [1]:
import pandas as pd
import re
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from scipy.interpolate import interp1d
import unidecode

# Funciones

In [4]:
# Normaliza el texto: convierte a minúsculas y elimina acentos
def normalize_text(text):
    if isinstance(text, str):
        return unidecode.unidecode(text.lower())
    return text

# Elimina los espacios al principio y al final del texto
def strip_text(text):
    if isinstance(text, str):
        return text.strip()
    return text

# Elimina todos los dígitos (números) de un texto
def remove_digits(text):
    if isinstance(text, str):
        return re.sub(r'\d+', '', text)
    return text

# Extrae únicamente los dígitos (números) de un texto y los convierte a entero
def extract_digits(text):
    if isinstance(text, str):
        return int(re.sub(r'\D', '', text))
    return text

In [5]:
# Interpolación y extrapolación de indicadores para un municipio específico
def interpolate_extrapolate_all_indicators(df, entidad, municipio, years_to_predict):
    # Filtrar los datos para la entidad y el municipio específicos
    df_municipio = df[(df['Municipio'] == municipio) & (df['Entidad'] == entidad)]

    # Lista de todas las columnas de indicadores (suponemos que las columnas numéricas son indicadores)
    indicator_cols = df.columns.difference(['Entidad', 'Municipio', 'Año Censal'])

    result_list = []

    # Iteramos sobre cada indicador y realizamos la interpolación/extrapolación
    for indicator_col in indicator_cols:
        # Extraer los años y los valores del indicador
        x = df_municipio['Año Censal'].values
        y = df_municipio[indicator_col].values

        print(f"Procesando indicador: {indicator_col}")
        print(f"Años: {x}")
        print(f"Valores: {y}")

        # Crear la función de interpolación (con modo 'extrapolate')
        f = interp1d(x, y, kind='linear', fill_value='extrapolate', bounds_error=False)

        # Predecir para los años solicitados
        y_pred = f(years_to_predict)

        # Crear un DataFrame con las nuevas predicciones para este indicador
        result_df = pd.DataFrame({
            'Entidad': [entidad] * len(years_to_predict),
            'Municipio': [municipio] * len(years_to_predict),
            'Año Censal': years_to_predict,
            indicator_col: y_pred
        })
        result_list.append(result_df)

    # Concatenar los resultados de todos los indicadores para este municipio
    all_results_df = pd.concat(result_list, axis=1)

    # Eliminar columnas duplicadas generadas por la concatenación
    all_results_df = all_results_df.loc[:, ~all_results_df.columns.duplicated()]

    return all_results_df

# Actualizar el DataFrame original con los valores interpolados para todos los indicadores
def update_with_interpolation_all_indicators(df, years_to_predict):
    # Obtener una lista de municipios y entidades únicas
    municipios = df[['Entidad', 'Municipio']].drop_duplicates()

    # Lista vacía para recolectar resultados
    all_results = []

    # Iterar sobre cada municipio y entidad
    for _, row in municipios.iterrows():
        print(f"Procesando municipio: {row['Municipio']}, Entidad: {row['Entidad']}")
        entidad = row['Entidad']
        municipio = row['Municipio']

        # Interpolar para cada municipio y todos los indicadores
        interpolated_data = interpolate_extrapolate_all_indicators(df, entidad, municipio, years_to_predict)

        # Agregar los datos interpolados a la lista de resultados
        all_results.append(interpolated_data)

    # Concatenar todos los resultados interpolados en un único DataFrame
    interpolated_df = pd.concat(all_results, ignore_index=True)

    # Combinar los datos interpolados con el DataFrame original
    df_updated = pd.concat([df, interpolated_df], ignore_index=True).sort_values(by=['Entidad', 'Municipio', 'Año Censal'])

    return df_updated

# Procesamiento de Datos Independientes


## Censo de Población


In [ ]:
ruta_censo_pob = "/content/drive/MyDrive/Dataton/Datos/INEGI/CensoPob.csv"
censo_pob = pd.read_csv(ruta_censo_pob)

# Filtrar datos a partir del año 2000 y ordenar por año, entidad y municipio
censo_pob = censo_pob[censo_pob['año'] >= 2000].sort_values(by=['año', 'cve_entidad', 'cve_municipio'])

# Filtrar para incluir solo entidades y municipios con claves mayores a 0
censo_pob = censo_pob[(censo_pob['cve_entidad'] > 0) & (censo_pob['cve_municipio'] > 0)]

# Contar el número de indicadores únicos por año
indicadores_por_año = censo_pob.groupby('año')['indicador'].nunique()
print(indicadores_por_año)

# Años de interés
años = [2000, 2005, 2010, 2020]
indicadores_por_año = []

# Obtener los indicadores únicos para cada año
for año in años:
    indicadores_por_año.append(set(censo_pob[censo_pob['año'] == año]['indicador'].unique()))

# Calcular la intersección de indicadores entre todos los años
interseccion_indicadores = set.intersection(*indicadores_por_año)

print(f"El número de indicadores en la intersección es: {len(interseccion_indicadores)}")

# Imprimir los indicadores de la intersección que no contienen ciertas palabras clave
for ind in sorted(interseccion_indicadores):
    if "hombre" not in ind and "mujer" not in ind and "jefatura" not in ind and "indígena" not in ind:
        print(ind)


In [ ]:
# Lista de indicadores a filtrar
indicadores_a_filtrar = [
    "Edad mediana",
    "Grado promedio de escolaridad de la población de 15 y más años",
    "Hogares censales",
    "Población de 5 años y más que asiste a la escuela",
    "Población derechohabiente a servicios de salud",
    "Población en hogares censales",
    "Población en hogares familiares",
    "Población en hogares no familiares",
    "Población total",
    "Porcentaje de población de 15 a 29 años",
    "Porcentaje de población de 60 y más años",
    "Promedio de ocupantes en viviendas particulares habitadas",
    "Tamaño promedio de los hogares censales",
    "Total de viviendas particulares habitadas",
    "Viviendas particulares habitadas",
    "Viviendas particulares habitadas que disponen de agua de la red pública en el ámbito de la vivienda",
    "Viviendas particulares habitadas que disponen de computadora",
    "Viviendas particulares habitadas que disponen de drenaje",
    "Viviendas particulares habitadas que disponen de energía eléctrica",
    "Viviendas particulares habitadas que disponen de excusado o sanitario",
    "Viviendas particulares habitadas que disponen de lavadora"
]

# Filtrar el censo por los indicadores seleccionados
censo_filtrado = censo_pob[censo_pob['indicador'].isin(indicadores_a_filtrar)]

In [ ]:
# Columnas a usar en el pivoteo
columnas_pivote = ['año', 'desc_entidad', 'desc_municipio']

# Crear tabla pivotada con los indicadores como columnas
pob_transpuesta = censo_filtrado.pivot_table(
    index=['año', 'desc_entidad', 'desc_municipio'],
    columns='indicador',
    values='valor',
    aggfunc='first'
).reset_index()

# Rellenar valores faltantes con -1
pob_transpuesta.fillna(-1, inplace=True)

# Renombrar las columnas de entidad y municipio
pob_transpuesta.rename(columns={'desc_entidad': 'Entidad', 'desc_municipio': 'Municipio'}, inplace=True)

# Definir los municipios con datos completos para todos los años de interés
entidades_municipios_completos = entidad_municipio_years[entidad_municipio_years == len(años)].index.tolist()

# Filtrar la tabla transpuesta para mantener solo los municipios con datos completos
pob_transpuesta_completa = pob_transpuesta[
    pob_transpuesta.apply(lambda row: (row['Entidad'], row['Municipio']) in entidades_municipios_completos, axis=1)
]

print(f"Número de filas en pob_transpuesta_completa: {len(pob_transpuesta_completa)}")

# Eliminar filas con valores faltantes (-1)
pob_transpuesta_sin_faltantes = pob_transpuesta_completa[~(pob_transpuesta_completa == -1).any(axis=1)]
print(f"Número de filas después de eliminar filas con -1: {len(pob_transpuesta_sin_faltantes)}")

# Eliminar filas que contienen 'ND' (No Disponible)
pob_transpuesta_sin_faltantes_nd = pob_transpuesta_sin_faltantes[~pob_transpuesta_sin_faltantes.apply(lambda row: 'ND' in row.astype(str).values, axis=1)]
print(f"Número de filas después de eliminar filas con ND: {len(pob_transpuesta_sin_faltantes_nd)}")

# Asignar la tabla final a pob
pob = pob_transpuesta_sin_faltantes_nd

# Ordenar los datos por Entidad, Municipio y Año
pob.sort_values(by=['Entidad', 'Municipio', 'año'], inplace=True)

# Renombrar la columna 'año' a 'Año Censal'
pob.rename(columns={'año': 'Año Censal'}, inplace=True)

In [ ]:
pob

indicador,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,Promedio de ocupantes en viviendas particulares habitadas,Tamaño promedio de los hogares censales,Total de viviendas particulares habitadas,Viviendas particulares habitadas,Viviendas particulares habitadas que disponen de agua de la red pública en el ámbito de la vivienda,Viviendas particulares habitadas que disponen de computadora,Viviendas particulares habitadas que disponen de drenaje,Viviendas particulares habitadas que disponen de energía eléctrica,Viviendas particulares habitadas que disponen de excusado o sanitario,Viviendas particulares habitadas que disponen de lavadora
0,2000,Aguascalientes,Aguascalientes,22,8.7,147147,190247,406230,637834,628290,...,4.5,4.33,141671,140845,137832,23715,137075,138978,137870,113574
2453,2005,Aguascalientes,Aguascalientes,24,9.3,173948,214633,515439,709280,694814,...,4.2,4.08,172191,169361,165100,53003,165966,167358,166074,145690
4905,2010,Aguascalientes,Aguascalientes,25,9.81,201071,231177,614582,791370,771336,...,3.94,3.94,202059,201071,197515,81576,198667,200175,199394,170657
7359,2020,Aguascalientes,Aguascalientes,28,10.8373396799837,266778,254122,774363,945505,906271,...,3.54486919717232,3.54416406150432,266778,257719,265232,136923,265392,265785,265781,234193
1,2000,Aguascalientes,Asientos,18,6,7561,10664,12042,37581,37247,...,5.1,4.97,7345,7315,6589,99,5782,6856,5729,4506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9824,2020,Zacatecas,Villanueva,31,8.08143420150788,9045,7652,23598,31420,30130,...,3.47497785651019,3.47374239911553,9045,9001,8916,2206,8670,8956,8760,7982
2452,2000,Zacatecas,Zacatecas,23,9.4,29755,39612,78512,122206,119213,...,4.3,4.11,28372,28272,27048,5041,26871,27912,27042,22907
4904,2005,Zacatecas,Zacatecas,25,10,32907,40303,90692,128839,125769,...,4.1,3.92,32326,31722,31088,11440,30946,31279,30803,27916
7358,2010,Zacatecas,Zacatecas,27,10.5,35388,40982,104160,135206,131289,...,3.82,3.82,36234,35388,34596,17090,34810,35125,34893,31093


### Interpolacion para alinearse con datos del censo economico

In [ ]:
years_to_predict = [2003, 2008, 2013, 2018]
pob_int = update_with_interpolation_all_indicators(pob, years_to_predict)

Streaming output truncated to the last 5000 lines.
Years: [2000 2005 2010 2020]
Values: ['461' '523' '642' '668']
Processing indicator: Viviendas particulares habitadas que disponen de excusado o sanitario
Years: [2000 2005 2010 2020]
Values: ['382' '433' '602' '508']
Processing indicator: Viviendas particulares habitadas que disponen de lavadora
Years: [2000 2005 2010 2020]
Values: ['67' '165' '316' '407']
Processing municipality: Tetiz, Entity: Yucatán
Processing indicator: Edad mediana
Years: [2000 2005 2010 2020]
Values: ['20' '22' '24' '28']
Processing indicator: Grado promedio de escolaridad de la población de 15 y más años
Years: [2000 2005 2010 2020]
Values: ['3.5' '4.4' '5.1' '6.43626206196039']
Processing indicator: Hogares censales
Years: [2000 2005 2010 2020]
Values: ['908' '1101' '1173' '1485']
Processing indicator: Población de 5 años y más que asiste a la escuela
Years: [2000 2005 2010 2020]
Values: ['1152' '1445' '1145' '1282']
Processing indicator: Población derechohab

In [ ]:
poblacion = pob_int

,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,Promedio de ocupantes en viviendas particulares habitadas,Tamaño promedio de los hogares censales,Total de viviendas particulares habitadas,Viviendas particulares habitadas,Viviendas particulares habitadas que disponen de agua de la red pública en el ámbito de la vivienda,Viviendas particulares habitadas que disponen de computadora,Viviendas particulares habitadas que disponen de drenaje,Viviendas particulares habitadas que disponen de energía eléctrica,Viviendas particulares habitadas que disponen de excusado o sanitario,Viviendas particulares habitadas que disponen de lavadora
0,2000,Aguascalientes,Aguascalientes,22,8.7,147147,190247,406230,637834,628290,...,4.5,4.33,141671,140845,137832,23715,137075,138978,137870,113574
9793,2003,Aguascalientes,Aguascalientes,23.2,9.06,163227.6,204878.6,471755.4,680701.6,668204.4,...,4.32,4.18,159983.0,157954.6,154192.8,41287.8,154409.6,156006.0,154792.4,132843.6
1,2005,Aguascalientes,Aguascalientes,24,9.3,173948,214633,515439,709280,694814,...,4.2,4.08,172191,169361,165100,53003,165966,167358,166074,145690
9794,2008,Aguascalientes,Aguascalientes,24.6,9.606,190221.8,224559.4,574924.8,758534.0,740727.2,...,4.044,3.996,190111.8,188387.0,184549.0,70146.8,185586.6,187048.2,186066.0,160670.2
2,2010,Aguascalientes,Aguascalientes,25,9.81,201071,231177,614582,791370,771336,...,3.94,3.94,202059,201071,197515,81576,198667,200175,199394,170657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24501,2013,Zacatecas,Zacatecas,27.9,10.819815,37489.8,41789.3,110105.7,138997.7,133917.3,...,3.720332,3.720221,38082.0,37171.2,36772.8,19193.0,37003.3,37260.7,37068.6,33111.7
24502,2015,Zacatecas,Zacatecas,28.5,11.033024,38891.0,42327.5,114069.5,141525.5,135669.5,...,3.653887,3.653702,39314.0,38360.0,38224.0,20595.0,38465.5,38684.5,38519.0,34457.5
24503,2018,Zacatecas,Zacatecas,29.4,11.352839,40992.8,43134.8,120015.2,145317.2,138297.8,...,3.55422,3.553923,41162.0,40143.2,40400.8,22698.0,40658.8,40820.2,40694.6,36476.2
9792,2020,Zacatecas,Zacatecas,30,11.5660489251297,42394,43673,123979,147845,140050,...,3.48777494571887,3.48740387790725,42394,41332,41852,24100,42121,42244,42145,37822


###  Extrapolacion 2024

In [ ]:
entidad_municipio_years = pob.groupby(['Entidad', 'Municipio'])['Año Censal'].nunique()

missing_years_entidad_municipio = entidad_municipio_years[entidad_municipio_years != len(pob['Año Censal'].unique())].reset_index()

complete_data_entidad_municipio = entidad_municipio_years[entidad_municipio_years == len(pob['Año Censal'].unique())].index.tolist()

pob2024 = pob[
    pob.apply(lambda row: (row['Entidad'], row['Municipio']) in complete_data_entidad_municipio, axis=1)
]

In [ ]:
years_to_predict = [2024]
pob2024 = update_with_interpolation_all_indicators(pob2024, years_to_predict)

## Comercio

In [ ]:
# Definir la ruta al archivo CSV del censo económico del comercio
ruta_comercio = "/content/drive/MyDrive/Dataton/Datos/INEGI/CensoEcoComercio.csv"
censo_comercio = pd.read_csv(ruta_comercio)

# Filtrar el dataset para incluir solo el 'Sector 46 Comercio al por menor' y eliminar valores faltantes
censo_comercio = censo_comercio[censo_comercio['Actividad económica'] == 'Sector 46 Comercio al por menor'].dropna()

# Aplicar la función para extraer los códigos de municipio y entidad
censo_comercio['cve_municipio'] = censo_comercio['Municipio'].apply(extract_digits)
censo_comercio['cve_entidad'] = censo_comercio['Entidad'].apply(extract_digits)

# Filtrar los datos para mantener solo municipios y entidades con claves válidas (> 0)
comercio = censo_comercio[(censo_comercio['cve_entidad'] > 0) & (censo_comercio['cve_municipio'] > 0)]

# Ordenar los datos por Año Censal, Entidad y Municipio
comercio.sort_values(by=['Año Censal', 'Entidad', 'Municipio'], inplace=True)

# Eliminar las columnas de 'Actividad económica', 'cve_municipio' y 'cve_entidad' y eliminar valores faltantes
comercio = comercio.drop(columns=['Actividad económica', 'cve_municipio', 'cve_entidad']).dropna()

# Aplicar la función remove_digits para limpiar los nombres de Entidad y Municipio
comercio['Entidad'] = comercio['Entidad'].apply(remove_digits)
comercio['Municipio'] = comercio['Municipio'].apply(remove_digits)

# Aplicar la función strip_text para eliminar espacios en blanco en los nombres de Entidad y Municipio
comercio['Entidad'] = comercio['Entidad'].apply(strip_text)
comercio['Municipio'] = comercio['Municipio'].apply(strip_text)



Posiblemente meter mas info de la economia y renumeraciones da cada sector para saber cuanto dinero tienen

In [ ]:
comercio

,Año Censal,Entidad,Municipio,UE Unidades económicas,H010A Personal remunerado total,J000A Total de remuneraciones (millones de pesos),K000A Total de gastos por consumo de bienes y servicios (millones de pesos),M000A Total de ingresos por suministro de bienes y servicios (millones de pesos),A111A Producción bruta total (millones de pesos),A121A Consumo intermedio (millones de pesos),A131A Valor agregado censal bruto (millones de pesos),A211A Inversión total (millones de pesos),A800A Total de ingresos (millones de pesos),A700A Total de gastos (millones de pesos)
14753,2003,Aguascalientes,Aguascalientes,11921,20043.0,928.782,15737.221,18402.716,4624.889,1654.141,2970.748,328.883,18482.622,15887.813
14755,2003,Aguascalientes,Asientos,225,82.0,2.991,43.020,57.921,18.273,2.970,15.303,1.294,57.921,43.182
14759,2003,Aguascalientes,Cosío,110,24.0,0.639,47.071,53.243,9.890,2.724,7.166,1.496,53.287,47.122
14761,2003,Aguascalientes,Jesús María,964,405.0,14.728,417.793,530.028,168.804,22.130,146.674,6.925,530.714,418.503
14763,2003,Aguascalientes,Pabellón de Arteaga,582,292.0,9.475,209.298,262.783,72.023,14.713,57.310,5.249,263.076,210.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,2018,Zacatecas,Vetagrande,82,8.0,0.268,13.401,17.654,5.500,1.132,4.368,0.154,17.667,13.434
4915,2018,Zacatecas,Villa de Cos,479,228.0,14.864,364.171,461.894,121.218,24.443,96.775,0.472,462.013,367.510
4917,2018,Zacatecas,Villa García,225,71.0,3.107,65.274,86.134,27.749,6.310,21.439,0.626,86.172,65.521
4923,2018,Zacatecas,Villanueva,597,317.0,20.827,489.804,588.989,132.759,35.231,97.528,3.693,593.344,493.471


### Extrapolacion 2024

In [ ]:
entidad_municipio_years = comercio.groupby(['Entidad', 'Municipio'])['Año Censal'].nunique()

missing_years_entidad_municipio = entidad_municipio_years[entidad_municipio_years != len(comercio['Año Censal'].unique())].reset_index()

print("Entidad and Municipio combinations missing data for all years:")
missing_years_entidad_municipio


Entidad and Municipio combinations missing data for all years:


,Entidad,Municipio,Año Censal
0,Aguascalientes,Calvillo,3
1,Campeche,Tenabo,2
2,Chiapas,Capitán Luis Ángel Vidal,1
3,Chiapas,Chamula,3
4,Chiapas,Copainalá,3
...,...,...,...
190,Zacatecas,Sain Alto,3
191,Zacatecas,Santa María de la Paz,3
192,Zacatecas,Trancoso,3
193,Zacatecas,Villa González Ortega,3


In [ ]:

entidad_municipio_years = comercio.groupby(['Entidad', 'Municipio'])['Año Censal'].nunique()

# Identify combinations with missing years
missing_years_entidad_municipio = entidad_municipio_years[entidad_municipio_years != len(comercio['Año Censal'].unique())].reset_index()

# Create a list of Entidad and Municipio combinations that have data for all years
complete_data_entidad_municipio = entidad_municipio_years[entidad_municipio_years == len(comercio['Año Censal'].unique())].index.tolist()

# Filter the DataFrame to keep only the rows with complete data for all years
com2024 = comercio[
    comercio.apply(lambda row: (row['Entidad'], row['Municipio']) in complete_data_entidad_municipio, axis=1)
]


In [ ]:
years_to_predict = [2024]
com2024 = update_with_interpolation_all_indicators(com2024, years_to_predict)

Streaming output truncated to the last 5000 lines.
Values: [15 12 20 35]
Processing municipality: Magdalena Teitipac, Entity: Oaxaca
Processing indicator: A111A Producción bruta total (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [2.124 2.591 1.458 6.873]
Processing indicator: A121A Consumo intermedio (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.313 0.744 0.404 0.85 ]
Processing indicator: A131A Valor agregado censal bruto (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [1.811 1.847 1.054 6.023]
Processing indicator: A211A Inversión total (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.666 0.117 0.053 0.052]
Processing indicator: A700A Total de gastos (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 3.275  7.003  4.962 17.311]
Processing indicator: A800A Total de ingresos (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 4.119  8.852  5.997 23.328]
Processing indicator: H010A Personal remunerado total
Years: [2003 2008 2

/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_interpolate.py:712: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


Processing municipality: San Juan Quiahije, Entity: Oaxaca
Processing indicator: A111A Producción bruta total (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.271 3.673 7.233 3.767]
Processing indicator: A121A Consumo intermedio (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.032 0.317 0.257 0.351]
Processing indicator: A131A Valor agregado censal bruto (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.239 3.356 6.976 3.416]
Processing indicator: A211A Inversión total (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 0.017 -0.006  0.317  0.148]
Processing indicator: A700A Total de gastos (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 1.064  4.802  8.022 14.266]
Processing indicator: A800A Total de ingresos (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 1.288  8.164 14.657 17.544]
Processing indicator: H010A Personal remunerado total
Years: [2003 2008 2013 2018]
Values: [0. 0. 6. 8.]
Processing indicator: J000A Total de r

/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_interpolate.py:712: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


Streaming output truncated to the last 5000 lines.
Years: [2003 2008 2013 2018]
Values: [150 208 253 244]
Processing municipality: Baca, Entity: Yucatán
Processing indicator: A111A Producción bruta total (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 4.117  5.061  7.332 20.284]
Processing indicator: A121A Consumo intermedio (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.752 1.829 2.47  4.122]
Processing indicator: A131A Valor agregado censal bruto (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [ 3.365  3.232  4.862 16.162]
Processing indicator: A211A Inversión total (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [0.482 0.367 0.175 3.027]
Processing indicator: A700A Total de gastos (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [10.962 20.308 24.903 66.748]
Processing indicator: A800A Total de ingresos (millones de pesos)
Years: [2003 2008 2013 2018]
Values: [13.798 23.062 29.642 80.602]
Processing indicator: H010A Personal remunera

## Hospitales

In [ ]:
ruta_hospitales = "/content/drive/MyDrive/Dataton/Datos/INEGI/CensoEcoHospitales.csv"
censo_hospitales = pd.read_csv(ruta_hospitales)

# Mostrar todas las actividades económicas presentes en el censo de hospitales
print(censo_hospitales['Actividad económica'].unique())

# Filtrar los datos que pertenecen al Sector 62 (Servicios de salud y asistencia social)
hospitales_sub = censo_hospitales[censo_hospitales['Actividad económica'].str.contains('Sector 62|Subsector 62')]

# Ordenar los datos por Año Censal, Entidad y Municipio
hospitales_sub.sort_values(by=['Año Censal', 'Entidad', 'Municipio'], inplace=True)

# Contar cuántas veces aparece cada actividad económica dentro del sector seleccionado
subsector_counts = hospitales_sub['Actividad económica'].value_counts()
print(subsector_counts)

In [ ]:
# Crear una tabla pivote con las Unidades Económicas (UE) agrupadas por año, entidad y municipio
UE_pivot = hospitales_sub.pivot_table(
    index=['Año Censal', 'Entidad', 'Municipio'],
    columns='Actividad económica',
    values='UE Unidades económicas',
    aggfunc='sum'
).reset_index()

# Rellenar los valores faltantes con 0
UE_pivot.fillna(0, inplace=True)

# Asignar el resultado de la tabla pivote a la variable hospitales
hospitales = UE_pivot

# Renombrar las columnas de la tabla pivote para mayor claridad
new_columns = {
    'Sector 62 Servicios de salud y de asistencia social': 'UE Sector 62 Servicios de salud y de asistencia social',
    'Subsector 621 Servicios médicos de consulta externa y servicios relacionados': 'UE Subsector 621 Servicios médicos de consulta externa y servicios relacionados',
    'Subsector 622 Hospitales': 'UE Subsector 622 Hospitales',
    'Subsector 623 Residencias de asistencia social y para el cuidado de la salud': 'UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud',
}

# Aplicar los nuevos nombres de columnas
hospitales.rename(columns=new_columns, inplace=True)

['Total municipal' 'Sector 62 Servicios de salud y de asistencia social'
 'Subsector 621 Servicios médicos de consulta externa y servicios relacionados'
 'Rama 6211 Consultorios médicos' 'Rama 6212 Consultorios dentales'
 'Rama 6213 Otros consultorios para el cuidado de la salud'
 'Rama 6214 Centros para la atención de pacientes que no requieren hospitalización'
 'Rama 6215 Laboratorios médicos y de diagnóstico'
 'Rama 6216 Servicios de enfermería a domicilio'
 'Rama 6219 Servicios de ambulancias, de bancos de órganos y otros servicios auxiliares al tratamiento médico'
 'Subsector 622 Hospitales' 'Rama 6221 Hospitales generales'
 'Subrama 62211 Hospitales generales'
 'Clase 622111 Hospitales generales del sector privado'
 'Clase 622211 Hospitales psiquiátricos y para el tratamiento por adicción del sector privado'
 'Subsector 623 Residencias de asistencia social y para el cuidado de la salud'
 'Rama 6231 Residencias con cuidados de enfermeras para enfermos convalecientes, en rehabilita

In [ ]:
# Extraer los dígitos de los campos de Entidad y Municipio
hospitales['cve_municipio'] = hospitales['Municipio'].apply(extract_digits)
hospitales['cve_entidad'] = hospitales['Entidad'].apply(extract_digits)

# Filtrar las filas donde las claves de entidad y municipio sean válidas (> 0)
hospitales = hospitales[(hospitales['cve_entidad'] > 0) & (hospitales['cve_municipio'] > 0)]

# Ordenar los datos por Año Censal, Entidad y Municipio
hospitales.sort_values(by=['Año Censal', 'Entidad', 'Municipio'], inplace=True)

# Eliminar las columnas de claves de municipio y entidad, así como las filas con valores nulos
hospitales = hospitales.drop(columns=['cve_municipio', 'cve_entidad']).dropna()

# Limpiar los nombres de Entidad y Municipio, eliminando dígitos y espacios en blanco
hospitales['Entidad'] = hospitales['Entidad'].apply(remove_digits).apply(strip_text)
hospitales['Municipio'] = hospitales['Municipio'].apply(remove_digits).apply(strip_text)

# Mostrar el DataFrame final
hospitales

### Extrapolacion 2024

In [ ]:
entidad_municipio_years = hospitales.groupby(['Entidad', 'Municipio'])['Año Censal'].nunique()

# Identify combinations with missing years
missing_years_entidad_municipio = entidad_municipio_years[entidad_municipio_years != len(hospitales['Año Censal'].unique())].reset_index()

# Create a list of Entidad and Municipio combinations that have data for all years
complete_data_entidad_municipio = entidad_municipio_years[entidad_municipio_years == len(hospitales['Año Censal'].unique())].index.tolist()

# Filter the DataFrame to keep only the rows with complete data for all years
hos2024 = hospitales[
    hospitales.apply(lambda row: (row['Entidad'], row['Municipio']) in complete_data_entidad_municipio, axis=1)
]

Actividad económica,Año Censal,Entidad,Municipio,UE Sector 62 Servicios de salud y de asistencia social,UE Subsector 621 Servicios médicos de consulta externa y servicios relacionados,UE Subsector 622 Hospitales,UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud
0,2003,Aguascalientes,Aguascalientes,1209.0,1084.0,21.0,8.0
1,2003,Aguascalientes,Asientos,9.0,6.0,0.0,1.0
2,2003,Aguascalientes,Calvillo,49.0,40.0,1.0,0.0
3,2003,Aguascalientes,Cosío,1.0,0.0,0.0,0.0
4,2003,Aguascalientes,Jesús María,42.0,30.0,0.0,0.0
...,...,...,...,...,...,...,...
7578,2018,Zacatecas,Villa González Ortega,13.0,9.0,0.0,2.0
7579,2018,Zacatecas,Villa Hidalgo,17.0,15.0,0.0,0.0
7580,2018,Zacatecas,Villanueva,35.0,33.0,0.0,0.0
7581,2018,Zacatecas,Zacatecas,541.0,458.0,2.0,3.0


In [ ]:
years_to_predict = [2024]
hos2024 = update_with_interpolation_all_indicators(hos2024, years_to_predict)

Streaming output truncated to the last 5000 lines.
Years: [2003 2008 2013 2018]
Values: [3. 6. 5. 5.]
Processing indicator: UE Subsector 622 Hospitales
Years: [2003 2008 2013 2018]
Values: [0. 0. 0. 0.]
Processing indicator: UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud
Years: [2003 2008 2013 2018]
Values: [0. 0. 0. 0.]
Processing municipality: Yécora, Entity: Sonora
Processing indicator: UE Sector 62 Servicios de salud y de asistencia social
Years: [2003 2008 2013 2018]
Values: [4. 3. 3. 3.]
Processing indicator: UE Subsector 621 Servicios médicos de consulta externa y servicios relacionados
Years: [2003 2008 2013 2018]
Values: [4. 3. 3. 3.]
Processing indicator: UE Subsector 622 Hospitales
Years: [2003 2008 2013 2018]
Values: [0. 0. 0. 0.]
Processing indicator: UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud
Years: [2003 2008 2013 2018]
Values: [0. 0. 0. 0.]
Processing municipality: General Plutarco Elías Calles, Entit

## Datos Geograficos

In [ ]:
# Importar la librería geopandas para trabajar con datos geoespaciales
import geopandas as gpd

# Definir la ruta al archivo shapefile de municipios
ruta_shapefile = '/content/drive/MyDrive/Dataton/Datos/INEGI/Municipios/mun21cw.shp'
municipios = gpd.read_file(ruta_shapefile)

# Mostrar los datos de los municipios
municipios

# Agrupar los datos por 'NOM_ENT' (nombre de entidad) y 'NOM_MUN' (nombre de municipio) y calcular el área total
total_area_por_municipio = municipios.groupby(['NOM_ENT', 'NOM_MUN'])['geometry'].apply(lambda x: x.unary_union.area).reset_index()

# Renombrar la columna del área para mayor claridad
total_area_por_municipio = total_area_por_municipio.rename(columns={'geometry': 'area_total'})

# Mostrar el DataFrame con el área total calculada
total_area_por_municipio

# Decodificar correctamente los nombres de entidad y municipio usando la codificación 'latin-1' y decodificarlos a 'utf-8'
total_area_por_municipio['NOM_ENT'] = total_area_por_municipio['NOM_ENT'].str.encode('latin-1').str.decode('utf-8')
total_area_por_municipio['NOM_MUN'] = total_area_por_municipio['NOM_MUN'].str.encode('latin-1').str.decode('utf-8')

# Mostrar el DataFrame final con nombres correctamente decodificados y el área total
total_area_por_municipio


## Merge

In [ ]:
# Normalizar los nombres de las entidades y municipios en los distintos DataFrames
comercio['Entidad'] = comercio['Entidad'].apply(normalize_text)
comercio['Municipio'] = comercio['Municipio'].apply(normalize_text)

pob['Entidad'] = pob['Entidad'].apply(normalize_text)
pob['Municipio'] = pob['Municipio'].apply(normalize_text)

total_area_by_mun['NOM_ENT'] = total_area_by_mun['NOM_ENT'].apply(normalize_text)
total_area_by_mun['NOM_MUN'] = total_area_by_mun['NOM_MUN'].apply(normalize_text)

hospitales['Entidad'] = hospitales['Entidad'].apply(normalize_text)
hospitales['Municipio'] = hospitales['Municipio'].apply(normalize_text)

# Comparar los municipios entre comercio y pob para encontrar diferencias por entidad
for entidad in comercio['Entidad'].unique():
    # Obtener los municipios únicos para cada entidad en comercio y pob
    municipios_comercio = set(comercio[comercio['Entidad'] == entidad]['Municipio'].unique())
    municipios_pob = set(pob[pob['Entidad'] == entidad]['Municipio'].unique())

    # Encontrar las diferencias entre municipios en comercio y pob
    diff_comercio = municipios_comercio - municipios_pob
    diff_pob = municipios_pob - municipios_comercio

    # Imprimir los municipios que están en comercio pero no en pob
    if diff_comercio:
        print(f"Municipios en Comercio que no están en Pob para la entidad {entidad}: {diff_comercio}")

    # Imprimir los municipios que están en pob pero no en comercio
    if diff_pob:
        print(f"Municipios en Pob que no están en Comercio para la entidad {entidad}: {diff_pob}")


In [ ]:
pob.head()

,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,Promedio de ocupantes en viviendas particulares habitadas,Tamaño promedio de los hogares censales,Total de viviendas particulares habitadas,Viviendas particulares habitadas,Viviendas particulares habitadas que disponen de agua de la red pública en el ámbito de la vivienda,Viviendas particulares habitadas que disponen de computadora,Viviendas particulares habitadas que disponen de drenaje,Viviendas particulares habitadas que disponen de energía eléctrica,Viviendas particulares habitadas que disponen de excusado o sanitario,Viviendas particulares habitadas que disponen de lavadora
0,2000,aguascalientes,aguascalientes,22,8.7,147147,190247,406230,637834,628290,...,4.5,4.33,141671,140845,137832,23715,137075,138978,137870,113574
9793,2003,aguascalientes,aguascalientes,23.2,9.06,163227.6,204878.6,471755.4,680701.6,668204.4,...,4.32,4.18,159983.0,157954.6,154192.8,41287.8,154409.6,156006.0,154792.4,132843.6
1,2005,aguascalientes,aguascalientes,24,9.3,173948,214633,515439,709280,694814,...,4.2,4.08,172191,169361,165100,53003,165966,167358,166074,145690
9794,2008,aguascalientes,aguascalientes,24.6,9.606,190221.8,224559.4,574924.8,758534.0,740727.2,...,4.044,3.996,190111.8,188387.0,184549.0,70146.8,185586.6,187048.2,186066.0,160670.2
2,2010,aguascalientes,aguascalientes,25,9.81,201071,231177,614582,791370,771336,...,3.94,3.94,202059,201071,197515,81576,198667,200175,199394,170657


In [ ]:
comercio.head()

,Año Censal,Entidad,Municipio,UE Unidades económicas,H010A Personal remunerado total,J000A Total de remuneraciones (millones de pesos),K000A Total de gastos por consumo de bienes y servicios (millones de pesos),M000A Total de ingresos por suministro de bienes y servicios (millones de pesos),A111A Producción bruta total (millones de pesos),A121A Consumo intermedio (millones de pesos),A131A Valor agregado censal bruto (millones de pesos),A211A Inversión total (millones de pesos),A800A Total de ingresos (millones de pesos),A700A Total de gastos (millones de pesos)
14753,2003,aguascalientes,aguascalientes,11921,20043.0,928.782,15737.221,18402.716,4624.889,1654.141,2970.748,328.883,18482.622,15887.813
14755,2003,aguascalientes,asientos,225,82.0,2.991,43.020,57.921,18.273,2.970,15.303,1.294,57.921,43.182
14757,2003,aguascalientes,calvillo,691,403.0,12.246,298.144,368.665,96.314,22.004,74.310,5.591,369.076,299.956
14759,2003,aguascalientes,cosio,110,24.0,0.639,47.071,53.243,9.890,2.724,7.166,1.496,53.287,47.122
14761,2003,aguascalientes,jesus maria,964,405.0,14.728,417.793,530.028,168.804,22.130,146.674,6.925,530.714,418.503


In [ ]:
hospitales.head()

Actividad económica,Año Censal,Entidad,Municipio,UE Sector 62 Servicios de salud y de asistencia social,UE Subsector 621 Servicios médicos de consulta externa y servicios relacionados,UE Subsector 622 Hospitales,UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud
0,2003,aguascalientes,aguascalientes,1209.0,1084.0,21.0,8.0
1,2003,aguascalientes,asientos,9.0,6.0,0.0,1.0
2,2003,aguascalientes,calvillo,49.0,40.0,1.0,0.0
3,2003,aguascalientes,cosio,1.0,0.0,0.0,0.0
4,2003,aguascalientes,jesus maria,42.0,30.0,0.0,0.0


In [ ]:
# Hacer un merge de los DataFrames pob, comercio, hospitales y total_area_by_mun
ind = pd.merge(pob, comercio, on=['Año Censal', 'Entidad', 'Municipio'], how='inner')
ind = pd.merge(ind, hospitales, on=['Año Censal', 'Entidad', 'Municipio'], how='inner')
ind = pd.merge(ind, total_area_by_mun, left_on=['Entidad', 'Municipio'], right_on=['NOM_ENT', 'NOM_MUN'], how='inner')

# Eliminar las columnas NOM_ENT y NOM_MUN después del merge
ind.drop(columns=['NOM_ENT', 'NOM_MUN'], inplace=True)

In [ ]:
ind

,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,A121A Consumo intermedio (millones de pesos),A131A Valor agregado censal bruto (millones de pesos),A211A Inversión total (millones de pesos),A800A Total de ingresos (millones de pesos),A700A Total de gastos (millones de pesos),UE Sector 62 Servicios de salud y de asistencia social,UE Subsector 621 Servicios médicos de consulta externa y servicios relacionados,UE Subsector 622 Hospitales,UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud,total_area
0,2003,aguascalientes,aguascalientes,23.2,9.06,163227.6,204878.6,471755.4,680701.6,668204.4,...,1654.141,2970.748,328.883,18482.622,15887.813,1209.0,1084.0,21.0,8.0,1.166359e+09
1,2008,aguascalientes,aguascalientes,24.6,9.606,190221.8,224559.4,574924.8,758534.0,740727.2,...,2544.220,3451.543,374.148,27679.637,24405.560,1726.0,1446.0,23.0,12.0,1.166359e+09
2,2013,aguascalientes,aguascalientes,25.9,10.118202,220783.1,238060.5,662516.3,837610.5,811816.5,...,3304.429,5162.113,417.154,36037.094,30852.028,2120.0,1791.0,18.0,26.0,1.166359e+09
3,2018,aguascalientes,aguascalientes,27.4,10.631872,253636.6,249533.0,742406.8,914678.0,879284.0,...,5379.096,12859.397,841.352,59412.051,47067.235,2464.0,2170.0,26.0,33.0,1.166359e+09
4,2003,aguascalientes,asientos,19.2,6.42,8324.8,11360.6,23247.0,39240.6,38817.8,...,2.970,15.303,1.294,57.921,43.182,9.0,6.0,0.0,1.0,5.433743e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7423,2018,zacatecas,villanueva,30.2,7.809147,8801.8,7636.6,22186.2,30990.4,29781.0,...,35.231,97.528,3.693,593.344,493.471,35.0,33.0,0.0,0.0,2.156506e+09
7424,2003,zacatecas,zacatecas,24.2,9.76,31646.2,40026.6,85820.0,126185.8,123146.6,...,284.611,710.392,39.003,3945.533,3272.023,310.0,278.0,3.0,0.0,4.368210e+08
7425,2008,zacatecas,zacatecas,26.2,10.3,34395.6,40710.4,98772.8,132659.2,129081.0,...,537.462,604.017,74.305,5429.802,4834.175,405.0,325.0,6.0,0.0,4.368210e+08
7426,2013,zacatecas,zacatecas,27.9,10.819815,37489.8,41789.3,110105.7,138997.7,133917.3,...,1308.294,1100.426,85.646,9144.038,8080.815,440.0,348.0,3.0,5.0,4.368210e+08


In [ ]:
# Filtrar las columnas que no son Entidad o Municipio para análisis posterior
columns_to_keep = [col for col in ind.columns if col not in ['Entidad', 'Municipio']]
value_cols = ind[columns_to_keep]

# Contar cuántas veces aparece el valor 0 en cada columna
zero_counts = (value_cols == 0).sum()
print(zero_counts)

Año Censal
Entidad
Municipio
Edad mediana
Grado promedio de escolaridad de la población de 15 y más años
Hogares censales
Población de 5 años y más que asiste a la escuela
Población derechohabiente a servicios de salud
Población en hogares censales
Población en hogares familiares
Población en hogares no familiares
Población total
Porcentaje de población de 15 a 29 años
Porcentaje de población de 60 y más años
Promedio de ocupantes en viviendas particulares habitadas
Tamaño promedio de los hogares censales
Total de viviendas particulares habitadas
Viviendas particulares habitadas
Viviendas particulares habitadas que disponen de agua de la red pública en el ámbito de la vivienda
Viviendas particulares habitadas que disponen de computadora
Viviendas particulares habitadas que disponen de drenaje
Viviendas particulares habitadas que disponen de energía eléctrica
Viviendas particulares habitadas que disponen de excusado o sanitario
Viviendas particulares habitadas que disponen de lavadora
U

### Indicadores Adicionales

In [ ]:
# Calculate production bruta per capita
ind['Prod_Bruta_Per_Capita'] = ind['A111A Producción bruta total (millones de pesos)'] * 1000000 / ind['Población total']

# Calculate percentage of population with health services
ind['%_Pob_Derechohabiente'] = (ind['Población derechohabiente a servicios de salud'] / ind['Población total']) * 100

# Calculate population density
ind['Densidad_Poblacional'] = (ind['Población total'] / ind['total_area'])*1000

# Display the updated DataFrame
ind[['Prod_Bruta_Per_Capita', '%_Pob_Derechohabiente', 'Densidad_Poblacional']]

,Prod_Bruta_Per_Capita,%_Pob_Derechohabiente,Densidad_Poblacional
0,6691.164875,68.252301,0.592608
1,7812.850849,74.916265,0.657965
2,10048.067657,78.627244,0.722422
3,19854.792215,80.819905,0.787574
4,463.388904,58.952563,0.072571
...,...,...,...
7423,4265.294583,71.280048,0.014433
7424,7726.342322,66.640472,0.294813
7425,8410.568555,72.777108,0.310698
7426,17010.097786,77.755352,0.324172


In [ ]:
rows_per_year = ind.groupby('Año Censal').size()
print("Rows per year:\n", rows_per_year)

municipios_per_year = ind.groupby('Año Censal')['Municipio'].nunique()
print("\nUnique municipios per year:\n", municipios_per_year)

entidades_per_year = ind.groupby('Año Censal')['Entidad'].nunique()
print("\nUnique entidades per year:\n", entidades_per_year)


Rows per year:
 Año Censal
2003    1692
2008    1876
2013    1908
2018    1952
dtype: int64

Unique municipios per year:
 Año Censal
2003    1597
2008    1773
2013    1800
2018    1836
Name: Municipio, dtype: int64

Unique entidades per year:
 Año Censal
2003    32
2008    32
2013    32
2018    32
Name: Entidad, dtype: int64


### Exportar

In [ ]:
ind.to_csv('/content/drive/MyDrive/Dataton/Datos/Procesados/Independientes.csv', index=False)

# Datos Farmacias

https://www.inegi.org.mx/app/descarga/?ti=6

## Comercio General

In [10]:
# Importar los datos del CSV correspondiente a farmacias
farms_data_path = "/content/drive/MyDrive/Dataton/Datos/INEGI/CensoEcoFarmacias.csv"
farms = pd.read_csv(farms_data_path).dropna()  # Leer el CSV y eliminar valores nulos

# Filtrar las filas que contienen 'minisúper' en la columna 'Actividad económica' (sin importar mayúsculas/minúsculas)
farms = farms[farms['Actividad económica'].str.contains('minisúper', na=False, case=False)]

# Agrupar por Año Censal, Entidad y Municipio, y sumar los valores numéricos
grouped_farms = farms.groupby(['Año Censal', 'Entidad', 'Municipio'], as_index=False).sum()

# Eliminar la columna 'Actividad económica' ya que no es relevante para el análisis posterior
grouped_farms.drop('Actividad económica', axis=1, inplace=True)

# Aplicar funciones de limpieza a las columnas de Entidad y Municipio
grouped_farms['Entidad'] = grouped_farms['Entidad'].apply(remove_digits).apply(strip_text).apply(normalize_text)
grouped_farms['Municipio'] = grouped_farms['Municipio'].apply(remove_digits).apply(strip_text).apply(normalize_text)

# Contar el número de valores cero en cada columna del DataFrame agrupado
zero_counts = (grouped_farms == 0).sum()
print("Número de ceros por columna:\n", zero_counts)


Número de ceros por columna:
 Año Censal                                                                                           0
Entidad                                                                                              0
Municipio                                                                                            0
UE Unidades económicas                                                                               0
A131A Valor agregado censal bruto (millones de pesos)                                                1
Q000A Acervo total de activos fijos (millones de pesos)                                              5
M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)                     0
A121A Consumo intermedio (millones de pesos)                                                         0
A700A Total de gastos (millones de pesos)                                                            0
Q000B Depreciación total de activos fijos (

### Exportar

In [ ]:
grouped_farms.to_csv('/content/drive/MyDrive/Dataton/Datos/Procesados/FarmDatos.csv', index=False)

## Conteo Unidades 2018

In [ ]:
# Importar el archivo CSV correspondiente a DENUE 2018
denue2018_path = "/content/drive/MyDrive/Dataton/Datos/INEGI/DENUE/2018.csv"
denue2018 = pd.read_csv(denue2018_path)

# Mostrar las primeras filas del DataFrame para ver la estructura de los datos
denue2018.head()

# Ver las columnas disponibles en el DataFrame
denue2018.columns

# Ver las actividades económicas únicas en la columna 'nombre_act'
denue2018['nombre_act'].unique()

# Agrupar por entidad y municipio y contar el número de unidades económicas por cada grupo
denue_counts = denue2018.groupby(['entidad', 'municipio'])['id'].count().reset_index()

# Renombrar la columna 'id' a 'num_units' para indicar que representa el número de unidades
denue_counts = denue_counts.rename(columns={'id': 'num_units'})

# Filtrar las filas donde la columna 'nombre_act' contenga 'Farmacias con minisúper' o 'Farmacias sin minisúper'
denue_filtered = denue2018[denue2018['nombre_act'].str.contains('Farmacias con minisúper|Farmacias sin minisúper', case=False, na=False)]

# Agrupar por entidad y municipio en los datos filtrados y contar el número de unidades
denue_municipio_counts = denue_filtered.groupby(['entidad', 'municipio'])['id'].count().reset_index()

# Renombrar la columna 'id' a 'num_units_municipio' para reflejar el número de unidades por municipio en los datos filtrados
denue_municipio_counts = denue_municipio_counts.rename(columns={'id': 'num_units_municipio'})

# Combinar los dos DataFrames: uno con los conteos generales y otro con los datos filtrados
denue = pd.merge(denue_counts, denue_municipio_counts, on=['entidad', 'municipio'], how='left')

# Agregar una columna indicando el año censal (2018)
denue['Año Censal'] = 2018

<ipython-input-278-e80c181bc82e>:2: DtypeWarning: Columns (14,18,34) have mixed types. Specify dtype option on import or set low_memory=False.
  denue2018 = pd.read_csv(denue2018_path)


In [ ]:
# Seleccionar las columnas relevantes del DataFrame 'grouped_farms' para el conteo de unidades económicas
farms_count = grouped_farms[['Año Censal', 'Entidad', 'Municipio', 'UE Unidades económicas']]

# Hacer un merge de 'farms_count' con el DataFrame 'denue' para obtener un conteo completo de unidades económicas
farm_counters = pd.merge(farms_count, denue, left_on=['Año Censal', 'Entidad', 'Municipio'], right_on=['Año Censal', 'entidad', 'municipio'], how='outer')

# Crear una nueva columna 'farms' con el valor máximo entre 'UE Unidades económicas' y 'num_units_municipio'
farm_counters['farms'] = farm_counters[['UE Unidades económicas', 'num_units_municipio']].max(axis=1)

# Seleccionar las columnas deseadas: 'Año Censal', 'Entidad', 'Municipio' y 'farms'
farm_count = farm_counters[['Año Censal', 'Entidad', 'Municipio', 'farms']]

# Eliminar las filas con valores faltantes
farm_count.dropna()

# Obtener los años únicos presentes en el DataFrame
unique_years = farm_count['Año Censal'].unique()
print(unique_years)

# Contar el número de municipios únicos por año
municipios_count = farm_count.groupby('Año Censal')['Municipio'].nunique()

# Contar el número de entidades únicas por año
entidades_count = farm_count.groupby('Año Censal')['Entidad'].nunique()

# Imprimir el número de municipios y entidades por año
print("Número de municipios por año:\n", municipios_count)
print("\nNúmero de entidades por año:\n", entidades_count)

# Hacer un merge de 'farm_count' con los ingresos totales de bienes y servicios
farm_count_with_ingresos = pd.merge(farm_count, grouped_farms[['Año Censal', 'Entidad', 'Municipio', 'M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)']],
                                    left_on=['Año Censal', 'Entidad', 'Municipio'],
                                    right_on=['Año Censal', 'Entidad', 'Municipio'],
                                    how='left')

# Renombrar la columna de ingresos para mayor claridad
farm_count_with_ingresos = farm_count_with_ingresos.rename(columns={'M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)': 'Total_Ingresos_Millones'}).dropna()

### Exportar

In [ ]:
farm_count_with_ingresos.to_csv('/content/drive/MyDrive/Dataton/Datos/Procesados/FarmCantidad.csv', index=False)

## Conteo Unidades 2024

In [7]:
# Importar el archivo CSV correspondiente a DENUE 2018
denue2024_path = "/content/drive/MyDrive/Dataton/Datos/INEGI/DENUE/2024.csv"
denue2024 = pd.read_csv(denue2024_path, encoding='latin-1')

# Mostrar las primeras filas del DataFrame para ver la estructura de los datos
denue2024.head()

# Ver las columnas disponibles en el DataFrame
denue2024.columns

# Ver las actividades económicas únicas en la columna 'nombre_act'
denue2024['nombre_act'].unique()

# Agrupar por entidad y municipio y contar el número de unidades económicas por cada grupo
denue_counts = denue2024.groupby(['entidad', 'municipio'])['id'].count().reset_index()

# Renombrar la columna 'id' a 'num_units' para indicar que representa el número de unidades
denue_counts = denue_counts.rename(columns={'id': 'num_units'})

# Filtrar las filas donde la columna 'nombre_act' contenga 'Farmacias con minisúper' o 'Farmacias sin minisúper'
denue_filtered = denue2024[denue2024['nombre_act'].str.contains('Farmacias con minisúper|Farmacias sin minisúper', case=False, na=False)]

# Agrupar por entidad y municipio en los datos filtrados y contar el número de unidades
denue_municipio_counts = denue_filtered.groupby(['entidad', 'municipio'])['id'].count().reset_index()

# Renombrar la columna 'id' a 'num_units_municipio' para reflejar el número de unidades por municipio en los datos filtrados
denue_municipio_counts = denue_municipio_counts.rename(columns={'id': 'num_units_municipio'})

# Combinar los dos DataFrames: uno con los conteos generales y otro con los datos filtrados
denue = pd.merge(denue_counts, denue_municipio_counts, on=['entidad', 'municipio'], how='left')

# Agregar una columna indicando el año censal (2018)
denue['Año Censal'] = 2024

<ipython-input-7-b767b9a57113>:3: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  denue2024 = pd.read_csv(denue2024_path, encoding='latin-1')


In [12]:
denue2024 = denue

In [16]:
denue2024['entidad'] = denue2024['entidad'].apply(remove_digits).apply(strip_text).apply(normalize_text)
denue2024['municipio'] = denue2024['municipio'].apply(remove_digits).apply(strip_text).apply(normalize_text)


In [17]:
denue2024

,entidad,municipio,num_units,num_units_municipio,Año Censal
0,aguascalientes,aguascalientes,4957,352.0,2024
1,aguascalientes,asientos,101,16.0,2024
2,aguascalientes,calvillo,274,16.0,2024
3,aguascalientes,cosio,42,6.0,2024
4,aguascalientes,el llano,34,2.0,2024
...,...,...,...,...,...
2369,zacatecas,villa gonzalez ortega,53,12.0,2024
2370,zacatecas,villa hidalgo,64,11.0,2024
2371,zacatecas,villa de cos,90,14.0,2024
2372,zacatecas,villanueva,184,14.0,2024


### Exportar

In [18]:
denue2024.to_csv('/content/drive/MyDrive/Dataton/Datos/Procesados/FarmCantidad2024.csv', index=False)